# Data Collection
## !!!!!!!!After running the notebook, delete the county_level folder inside data\data_raw\BEA_Industry_Factors!!!!!!!!!
Getting data from 2012 to 2016  
Years Prior to 2012 Currently have a bug when specifying NAICS#### as one of the columns
Years Prior to 2012 also only have the totals for all sectors not each sector.  
Years after 2017 not available  
Zip level not available -> https://api.census.gov/data/2011/cbp/variables.html  
ended up not using 2017 since the NAICS 2017 wasn't available in master_crrosswalk

In [1]:
import csv
import requests as r
import pandas as pd
import zipfile, io
import os
from tqdm import tqdm
import pathlib
import pandas as pd
api_headers = {}
api_headers['x-api-key'] = '975f39a54e48438ceebf303d6018e34db212e804'

In [2]:
# Set a relative location to save the data from the request
repo_dir = pathlib.Path().cwd()
raw_data_dir = repo_dir / 'data_raw'
out_data_dir = raw_data_dir / 'BEA_Industry_Factors'

# Create new folders if needed
#state_data_dir = out_data_dir / 'state_level'
#if not state_data_dir.exists():
#    state_data_dir.mkdir(parents=True)
    
county_data_dir = out_data_dir / 'county_level'
if not county_data_dir.exists():
    county_data_dir.mkdir(parents=True)
    
#zip_data_dir = out_data_dir / 'zip_level'
#if not county_data_dir.exists():
#    county_data_dir.mkdir(parents=True)

In [3]:
county_data_dir

WindowsPath('C:/Users/Owner/Documents/modelearth/community/info/data/data_raw/BEA_Industry_Factors/county_level')

In [4]:
# Load the state FIPS codes key
state_fips = pd.read_csv(out_data_dir / 'state_fips.csv', usecols=['Name', 'Postal Code', 'FIPS'])
state_fips = state_fips.head(50)  # <-- limit to only US states, not teritories

In [5]:
# Base URL for the API call
base_url = "https://api.census.gov/data"

#
# NOTE Years Prior to 2012 Currently have a bug when specifying NAICS#### as one of the columns
#      - stick to 2012 and later for now
#


def get_county_cbp(fips, state, years):
    count=0
    for year in years:
        print(f"Getting data for state: {state}\tyear: {year}")
        if year>=2000 and year<=2002:
            columns_to_select = "GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS1997_TTL,ESTAB,EMP,PAYANN"
            url=f"{base_url}/{year}/cbp?get={columns_to_select}&for=county:*&in=state:{fips:02d}"
        elif year>=2003 and year<=2007:
            columns_to_select = "GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS2002_TTL,ESTAB,EMP,PAYANN"
            url=f"{base_url}/{year}/cbp?get={columns_to_select}&for=county:*&in=state:{fips:02d}"
        elif year>=2008 and year<=2011:
            columns_to_select = "GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS2007_TTL,ESTAB,EMP,PAYANN"
            url=f"{base_url}/{year}/cbp?get={columns_to_select}&for=county:*&in=state:{fips:02d}"
        elif year>=2012 and year<=2016:
            columns_to_select = "GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS2012,NAICS2012_TTL,ESTAB,EMP,PAYANN"
            url=f"{base_url}/{year}/cbp?get={columns_to_select}&for=county:*&in=state:{fips:02d}"
        elif year==2017:
            columns_to_select = "GEO_ID,NAME,COUNTY,YEAR,NAICS2017,NAICS2017_LABEL,ESTAB,EMP,PAYANN"
            url=f"{base_url}/{year}/cbp?get={columns_to_select}&for=county:*&in=state:{fips:02d}"
    
    
        response = r.get(url, headers=api_headers)

        with open(county_data_dir / f"industriesPerCounty_{str.lower(state.replace(' ', ''))}_{year}.csv",'w') as resultPath:
            for line in response.text.strip().split('\n'):
                line=line.replace('[',"").replace(']',"")
                resultPath.write(line + "\n")

        print("  > Finished CSV for year"+str(year))

In [152]:
for fips in state_fips.FIPS.unique():
     state = state_fips.query(f'FIPS=={fips}').values[0][0]
     #years=range(2012,2018)
     years=range(2012,2017)
     get_county_cbp(fips, state, years)

Getting data for state: Alabama	year: 2008
  > Finished CSV for year2008
Getting data for state: Alabama	year: 2009
  > Finished CSV for year2009
Getting data for state: Alabama	year: 2010
  > Finished CSV for year2010
Getting data for state: Alabama	year: 2011
  > Finished CSV for year2011
Getting data for state: Alaska	year: 2008
  > Finished CSV for year2008
Getting data for state: Alaska	year: 2009
  > Finished CSV for year2009
Getting data for state: Alaska	year: 2010
  > Finished CSV for year2010
Getting data for state: Alaska	year: 2011
  > Finished CSV for year2011
Getting data for state: Arizona	year: 2008
  > Finished CSV for year2008
Getting data for state: Arizona	year: 2009
  > Finished CSV for year2009
Getting data for state: Arizona	year: 2010
  > Finished CSV for year2010
Getting data for state: Arizona	year: 2011
  > Finished CSV for year2011
Getting data for state: Arkansas	year: 2008
  > Finished CSV for year2008
Getting data for state: Arkansas	year: 2009
  > Finish

KeyboardInterrupt: 

# Data Aggregation
This part allows us to manage different Fips level (county/state) and different NAICS level (sector/industry/etc...)  
Trick: Getting the NAICS code from all the NAICS2012 files that we downloaded

In [8]:
# Load the 2012 data

def load_all_states2012(bea_data_dir):
    
    for i in range(2012,2017):
        #print(i)
        x="_"+str(i)
        files = [f for f in bea_data_dir.iterdir() if x in f.name]

        for f in files:
            #print(f)

            df = pd.read_csv(f,encoding='latin-1',dtype={"NAICS2012": str}).drop("Unnamed: 11", axis=1)
            #df = pd.read_csv(f)
            #df[df['NAICS2012'].str.len() != 5]
            #b=df[df['NAICS2012'].str.len() == 5]
            df['is5'] = df['NAICS2012'].apply(lambda x: 'True' if len(x) == 5 else 'False')
                #print("hh"+df.NAICS2012)
                #print("hghg"+len(df.NAICS2012))
            #df['NAICS_Sector'] = df['NAICS2012'].apply(lambda v: v[:2] if df['is5']=='False' else 1).apply(int)
            df.loc[(df['is5'] == 'True') & (df['NAICS2012'].apply(lambda v: v[2:3]) == '-'), 'NAICS_Sector'] = df['NAICS2012']
            df.loc[(df['is5'] == 'True') & (df['NAICS2012'].apply(lambda v: v[2:3]) != '-'), 'NAICS_Sector'] = df['NAICS2012'].apply(lambda v: v[:2])
            df.loc[(df['is5'] == 'False') , 'NAICS_Sector'] = df['NAICS2012'].apply(lambda v: v[:2])
                #print("hgh"+df.NAICS2012[1:2])
                #if (df.NAICS2012.apply(lambda v: v[0:1]).apply(int)==1).bool():
                #    print("bye")

            yield df
    
df = pd.concat(load_all_states2012(county_data_dir)).drop("is5", axis=1)

# test = list(load_all_states(bea_data_dir))

# df

In [9]:
df=df.drop("county", axis=1)

### Process FIPS Code
FIPS is the federal/census unique ID for each geographic area.  States have 2 digives and counties have 5

In [10]:
# Process FIPS code
df['fips'] = df.GEO_ID.apply(lambda GID: GID.split('US')[1])
# # Remove the state_level data
# df_state = df[df['fips'].str.len() == 2]
# df_county = df[df['fips'].str.len() != 2]

def county_level(df):
    return df[df['id'].str.len() == 5]

def state_level(df):
    return df[df['id'].str.len() == 2]

In [11]:
#df.to_csv("allll.csv")

### Group all data per sector for  every year and average them  
Does it make sense to sum up instead of average?!

In [12]:
newDF=df.groupby(['fips','state','COUNTY','GEO_TTL','NAICS2012','NAICS2012_TTL','NAICS_Sector'],as_index=False).mean()
newDF=newDF.drop("YEAR", axis=1)

In [13]:
#newDF=newDF.astype({'fips': 'int'})
newDF=newDF.rename(columns={"fips": "id", "NAICS2012": "relevant_naics","EMP":"emp","PAYANN":"payann","ESTAB":"estab"})
#b=b[["id","relevant_naics","emp","payann","estab"]]
newDF

,id,state,COUNTY,GEO_TTL,relevant_naics,NAICS2012_TTL,NAICS_Sector,estab,emp,payann
0,01,1,999,Alabama -- Statewide,00,Total for all sectors,00,405.80,42373.4,1333508.6
1,01,1,999,Alabama -- Statewide,21,"Mining, quarrying, and oil and gas extraction",21,18.00,386.4,27332.2
2,01,1,999,Alabama -- Statewide,211,Oil and gas extraction,21,4.20,31.6,3418.2
3,01,1,999,Alabama -- Statewide,2111,Oil and gas extraction,21,4.20,31.6,3418.2
4,01,1,999,Alabama -- Statewide,21111,Oil and gas extraction,21,4.20,31.6,3418.2
...,...,...,...,...,...,...,...,...,...,...
2365246,56045,56,45,"Weston County, Wyoming",81391,Business associations,81,1.00,0.0,0.0
2365247,56045,56,45,"Weston County, Wyoming",813910,Business associations,81,1.00,0.0,0.0
2365248,56045,56,45,"Weston County, Wyoming",81393,Labor unions and similar labor organizations,81,1.00,0.0,0.0
2365249,56045,56,45,"Weston County, Wyoming",813930,Labor unions and similar labor organizations,81,1.00,0.0,0.0


In [14]:
newDF.head(50)

,id,state,COUNTY,GEO_TTL,relevant_naics,NAICS2012_TTL,NAICS_Sector,estab,emp,payann
0,01,1,999,Alabama -- Statewide,00,Total for all sectors,00,405.800000,42373.4,1333508.6
1,01,1,999,Alabama -- Statewide,21,"Mining, quarrying, and oil and gas extraction",21,18.000000,386.4,27332.2
2,01,1,999,Alabama -- Statewide,211,Oil and gas extraction,21,4.200000,31.6,3418.2
3,01,1,999,Alabama -- Statewide,2111,Oil and gas extraction,21,4.200000,31.6,3418.2
4,01,1,999,Alabama -- Statewide,21111,Oil and gas extraction,21,4.200000,31.6,3418.2
5,01,1,999,Alabama -- Statewide,211111,Crude petroleum and natural gas extraction,21,4.200000,31.6,3418.2
6,01,1,999,Alabama -- Statewide,213,Support activities for mining,21,13.800000,236.6,13282.6
7,01,1,999,Alabama -- Statewide,2131,Support activities for mining,21,13.800000,236.6,13282.6
8,01,1,999,Alabama -- Statewide,21311,Support activities for mining,21,13.800000,236.6,13282.6
9,01,1,999,Alabama -- Statewide,213111,Drilling oil and gas wells,21,5.800000,48.4,1924.4


### Group data by NAICS Sector

NAICS is the North American Industry Classification System. The coarsest level of classification is the *Sector*.

The organization of NAICS is as follows:  <-- from [this page](https://www.census.gov/programs-surveys/economic-census/guidance/understanding-naics.html) on census.gov
- Sector: 2-digit code
    - Subsector: 3-digit code
        - Industry Group: 4-digit code
            - NAICS Industry: 5-digit code
                - National Industry: 6-digit code

Start by grouping the data by sector:

In [15]:
def naics_level(df, naics_level):
    return df[df['relevant_naics'].str.len() == naics_level]

In [16]:
#df.head()
# df_county['NAICS2012'].apply(lambda v: v.split('-')[0])

df_naics_2 = naics_level(newDF, 2).reset_index(drop=True)
df_naics_3 = naics_level(newDF, 3).reset_index(drop=True)
df_naics_4 = naics_level(newDF, 4).reset_index(drop=True)
df_naics_5 = naics_level(newDF, 5).reset_index(drop=True)
df_naics_6 = naics_level(newDF, 6).reset_index(drop=True)

# First remove all the per-county totals across all sectors
#df_totals = df_naics_2[df_naics_2.NAICS2012 == '00']
df_naics_2 = df_naics_2[df_naics_2.relevant_naics != '00']
df_naics_3 = df_naics_3[df_naics_3.relevant_naics != '00']
df_naics_4 = df_naics_4[df_naics_4.relevant_naics != '00']
df_naics_5 = df_naics_5[df_naics_5.relevant_naics != '00']
df_naics_6 = df_naics_6[df_naics_6.relevant_naics != '00']

In [17]:
#s2=state_level(df_naics_2)
c2=county_level(df_naics_2)
#s3=state_level(df_naics_3)
#c3=county_level(df_naics_3)
#s4=state_level(df_naics_4)
c4=county_level(df_naics_4)
#s5=state_level(df_naics_5)
#c5=county_level(df_naics_5)
s6=state_level(df_naics_6)
#c6=county_level(df_naics_6)

In [18]:
#s2.to_csv('s2.tsv',sep='\t')
#c2.to_csv('c2.tsv',sep='\t')
#s3.to_csv('s3.tsv',sep='\t')
#c3.to_csv('c3.tsv',sep='\t')
#s4.to_csv('s4.tsv',sep='\t')
#c4.to_csv('c4.tsv',sep='\t')
#s5.to_csv('s5.tsv',sep='\t')
#c5.to_csv('c5.tsv',sep='\t')
#s6.to_csv('s6.tsv',sep='\t')
#c6.to_csv('c6.tsv',sep='\t')

In [19]:
newDF

,id,state,COUNTY,GEO_TTL,relevant_naics,NAICS2012_TTL,NAICS_Sector,estab,emp,payann
0,01,1,999,Alabama -- Statewide,00,Total for all sectors,00,405.80,42373.4,1333508.6
1,01,1,999,Alabama -- Statewide,21,"Mining, quarrying, and oil and gas extraction",21,18.00,386.4,27332.2
2,01,1,999,Alabama -- Statewide,211,Oil and gas extraction,21,4.20,31.6,3418.2
3,01,1,999,Alabama -- Statewide,2111,Oil and gas extraction,21,4.20,31.6,3418.2
4,01,1,999,Alabama -- Statewide,21111,Oil and gas extraction,21,4.20,31.6,3418.2
...,...,...,...,...,...,...,...,...,...,...
2365246,56045,56,45,"Weston County, Wyoming",81391,Business associations,81,1.00,0.0,0.0
2365247,56045,56,45,"Weston County, Wyoming",813910,Business associations,81,1.00,0.0,0.0
2365248,56045,56,45,"Weston County, Wyoming",81393,Labor unions and similar labor organizations,81,1.00,0.0,0.0
2365249,56045,56,45,"Weston County, Wyoming",813930,Labor unions and similar labor organizations,81,1.00,0.0,0.0


In [90]:
#newDF.to_csv('all.tsv',sep='\t')

# The statewide data does not include NAICS starting with 1!

In [20]:
a=county_level(df_naics_2)

In [21]:
county_level(df_naics_6)

,id,state,COUNTY,GEO_TTL,relevant_naics,NAICS2012_TTL,NAICS_Sector,estab,emp,payann
156,01001,1,1,"Autauga County, Alabama",113310,Logging,11,6.0,54.8,2611.2
157,01001,1,1,"Autauga County, Alabama",115112,"Soil preparation, planting, and cultivating",11,1.0,0.0,0.0
158,01001,1,1,"Autauga County, Alabama",115310,Support activities for forestry,11,1.0,0.0,0.0
159,01001,1,1,"Autauga County, Alabama",212319,Other crushed and broken stone mining and quar...,21,1.0,0.0,0.0
160,01001,1,1,"Autauga County, Alabama",212321,Construction sand and gravel mining,21,1.6,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
892472,56045,56,45,"Weston County, Wyoming",813110,Religious organizations,81,8.6,10.6,226.8
892473,56045,56,45,"Weston County, Wyoming",813312,"Environment, conservation and wildlife organiz...",81,1.0,0.0,0.0
892474,56045,56,45,"Weston County, Wyoming",813319,Other social advocacy organizations,81,1.0,0.0,0.0
892475,56045,56,45,"Weston County, Wyoming",813910,Business associations,81,1.0,0.0,0.0


In [22]:
df_naics_5[df_naics_5.id=='13285']

,id,state,COUNTY,GEO_TTL,relevant_naics,NAICS2012_TTL,NAICS_Sector,estab,emp,payann
139527,13285,13,285,"Troup County, Georgia",11311,Timber tract operations,11,1.0,0.0,0.0
139528,13285,13,285,"Troup County, Georgia",11331,Logging,11,3.0,13.6,744.8
139529,13285,13,285,"Troup County, Georgia",11521,Support activities for animal production,11,1.0,0.0,0.0
139530,13285,13,285,"Troup County, Georgia",11531,Support activities for forestry,11,1.8,0.0,0.0
139531,13285,13,285,"Troup County, Georgia",21231,Stone mining and quarrying,21,1.2,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
139879,13285,13,285,"Troup County, Georgia",81321,Grantmaking and giving services,81,4.4,10.0,511.4
139880,13285,13,285,"Troup County, Georgia",81331,Social advocacy organizations,81,2.4,1.0,31.4
139881,13285,13,285,"Troup County, Georgia",81341,Civic and social organizations,81,1.0,0.0,0.0
139882,13285,13,285,"Troup County, Georgia",81391,Business associations,81,3.6,5.6,320.2


In [28]:
a=df_naics_6[df_naics_6.id=='13001'].sort_values(by=['payann'],ascending=False).head(50)
a.to_csv("investigate.csv")

# NAICS code to name translation

In [29]:
NAICS_codes = pd.read_csv(out_data_dir / 'Crosswalk_MasterCrosswalk.csv', usecols=['2012_NAICS_Code', '2012_NAICS_Industry'])

In [30]:
NAICS_codes=NAICS_codes.rename(columns={"2012_NAICS_Code": "relevant_naics", "2012_NAICS_Industry": "industry_detail"})

In [31]:
NAICS_codes=NAICS_codes.dropna()

In [32]:
NAICS_codes=NAICS_codes.drop_duplicates()

In [33]:
NAICS_codes

,relevant_naics,industry_detail
0,11.0,"Agriculture, Forestry, Fishing and Hunting"
13,111.0,Crop Production
19,1111.0,Oilseed and Grain Farming
21,11111.0,Soybean Farming
22,111110.0,Soybean Farming
...,...,...
3885,9281.0,National Security and International Affairs
3887,92811.0,National Security
3888,928110.0,National Security
3889,92812.0,International Affairs


In [34]:
#adding the row for Industries not classified
NAICS_codes
new_row = {'relevant_naics':99, 'industry_detail':"Industries not classified"}
#append row to the dataframe
NAICS_codes = NAICS_codes.append(new_row, ignore_index=True)

In [35]:
NAICS_codes

,relevant_naics,industry_detail
0,11.0,"Agriculture, Forestry, Fishing and Hunting"
1,111.0,Crop Production
2,1111.0,Oilseed and Grain Farming
3,11111.0,Soybean Farming
4,111110.0,Soybean Farming
...,...,...
2210,92811.0,National Security
2211,928110.0,National Security
2212,92812.0,International Affairs
2213,928120.0,International Affairs


In [26]:
NAICS_codes.to_csv('industry_ID_list.csv')

# Making a states json

In [36]:
stateFips = pd.read_csv(out_data_dir / 'state_fips.csv')

In [37]:
stateFips=stateFips.drop(['Unnamed: 3','Unnamed: 4','Unnamed: 5','Unnamed: 6'],axis=1)

In [38]:
#stateFips=stateFips.set_index('Name')

In [39]:
stateFips

,Name,Postal Code,FIPS
0,Alabama,AL,1
1,Alaska,AK,2
2,Arizona,AZ,4
3,Arkansas,AR,5
4,California,CA,6
5,Colorado,CO,8
6,Connecticut,CT,9
7,Delaware,DE,10
8,Florida,FL,12
9,Georgia,GA,13


In [31]:
stateFips.to_json('states.json', orient = "records", date_format = "epoch", double_precision = 10, force_ascii = True, date_unit = "ms", default_handler = None)

# Making county to fips csv

In [40]:
countyDF=c2[['GEO_TTL','id']].drop_duplicates()

In [41]:
countyDF

,GEO_TTL,id
16,"Autauga County, Alabama",01001
34,"Baldwin County, Alabama",01003
52,"Barbour County, Alabama",01005
70,"Bibb County, Alabama",01007
88,"Blount County, Alabama",01009
...,...,...
53916,"Sweetwater County, Wyoming",56037
53934,"Teton County, Wyoming",56039
53952,"Uinta County, Wyoming",56041
53969,"Washakie County, Wyoming",56043


In [42]:
countyDF['hascomma'] =countyDF['GEO_TTL'].apply(lambda x: 'True' if ',' in x else 'False')
countyDF
#countyDF['county'] = df.GEO_TTL.apply(lambda GTT: GTT.split(',')[0])

,GEO_TTL,id,hascomma
16,"Autauga County, Alabama",01001,True
34,"Baldwin County, Alabama",01003,True
52,"Barbour County, Alabama",01005,True
70,"Bibb County, Alabama",01007,True
88,"Blount County, Alabama",01009,True
...,...,...,...
53916,"Sweetwater County, Wyoming",56037,True
53934,"Teton County, Wyoming",56039,True
53952,"Uinta County, Wyoming",56041,True
53969,"Washakie County, Wyoming",56043,True


In [43]:
countyDF.loc[(countyDF['hascomma'] == 'True'), 'county'] = countyDF.GEO_TTL.apply(lambda GTT: GTT.split(', ')[0])
countyDF.loc[(countyDF['hascomma'] == 'True'), 'state'] = countyDF.GEO_TTL.apply(lambda GTT: GTT.split(', ')[1])

In [44]:
countyDF=countyDF[['state','county','id']].drop_duplicates()

In [45]:
countyDF

,state,county,id
16,Alabama,Autauga County,01001
34,Alabama,Baldwin County,01003
52,Alabama,Barbour County,01005
70,Alabama,Bibb County,01007
88,Alabama,Blount County,01009
...,...,...,...
53916,Wyoming,Sweetwater County,56037
53934,Wyoming,Teton County,56039
53952,Wyoming,Uinta County,56041
53969,Wyoming,Washakie County,56043


In [46]:
stats=stateFips.rename(columns={"Name": "state"})

In [47]:
stats=stats.drop("FIPS",axis=1)
stats

,state,Postal Code
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA
5,Colorado,CO
6,Connecticut,CT
7,Delaware,DE
8,Florida,FL
9,Georgia,GA


In [48]:
countyDF=countyDF.merge(stats, on='state', how='left')

In [49]:
countyDF=countyDF.rename(columns={"Postal Code": "abvr"})
countyDF

,state,county,id,abvr
0,Alabama,Autauga County,01001,AL
1,Alabama,Baldwin County,01003,AL
2,Alabama,Barbour County,01005,AL
3,Alabama,Bibb County,01007,AL
4,Alabama,Blount County,01009,AL
...,...,...,...,...
3136,Wyoming,Sweetwater County,56037,WY
3137,Wyoming,Teton County,56039,WY
3138,Wyoming,Uinta County,56041,WY
3139,Wyoming,Washakie County,56043,WY


In [44]:
countyDF.to_csv('county_ID_list.csv')

# saving the statewide data

In [51]:
states=newDF.state.unique()
#states=[13]

b1=county_level(df_naics_2)
c1=b1[b1.state==13]
c1.astype({'NAICS_Sector': 'int'})
d1=c1.groupby(['NAICS_Sector','NAICS2012_TTL','state','relevant_naics'],as_index=False).sum().drop("COUNTY",axis=1)
d1.insert(0, 'id', 13)
d1.insert(1, 'COUNTY', 999)
d1.insert(2, 'GEO_TTL', 'Statewide')

b2=county_level(df_naics_4)
c2=b2[b2.state==13]
c2.astype({'NAICS_Sector': 'int'})
d2=c2.groupby(['NAICS_Sector','NAICS2012_TTL','state','relevant_naics'],as_index=False).sum().drop("COUNTY",axis=1)
d2.insert(0, 'id', 13)
d2.insert(1, 'COUNTY', 999)
d2.insert(2, 'GEO_TTL', 'Statewide')

b3=county_level(df_naics_6)
c3=b3[b3.state==13]
c3.astype({'NAICS_Sector': 'int'})
d3=c3.groupby(['NAICS_Sector','NAICS2012_TTL','state','relevant_naics'],as_index=False).sum().drop("COUNTY",axis=1)
d3.insert(0, 'id', 13)
d3.insert(1, 'COUNTY', 999)
d3.insert(2, 'GEO_TTL', 'Statewide')

for state in states:
    stateName=stateFips.loc[stateFips.FIPS==state,"Postal Code"].values[0]
    print(stateName)
    a11=d1[d1.state==state]
    a11.to_csv(f"usa/{stateName}/industries_state{state}_naics2_state.tsv",sep='\t')
    #print(a11)
    #print(a11.dtypes)
    
    b11=d2[d2.state==state]
    b11.to_csv(f"usa/{stateName}/industries_state{state}_naics4_state.tsv",sep='\t')
    
    c11=d3[d3.state==state]
    c11.to_csv(f"usa/{stateName}/industries_state{state}_naics6_state.tsv",sep='\t')

GA


### statewide from API

In [46]:
states=newDF.state.unique()
#states=[13]
a=state_level(df_naics_2)
b=state_level(df_naics_4)
c=state_level(df_naics_6)
for state in states:
    stateName=stateFips.loc[stateFips.FIPS==state,"Postal Code"].values[0]
    print(stateName)
    a1=a[a.state==state]
    a1.to_csv(f"usa/{stateName}/industries_state{state}_naics2_state_api.tsv",sep='\t')
    #print(a1)
    #print(a1.dtypes)
    
    b1=b[b.state==state]
    b1.to_csv(f"usa/{stateName}/industries_state{state}_naics4_state_api.tsv",sep='\t')
    
    c1=c[c.state==state]
    c1.to_csv(f"usa/{stateName}/industries_state{state}_naics6_state_api.tsv",sep='\t')

AL
AK
AZ
AR
CA
CO
CT
DE
FL
GA
HI
ID
IL
IN
IA
KS
KY
LA
ME
MD
MA
MI
MN
MS
MO
MT
NE
NV
NH
NJ
NM
NY
NC
ND
OH
OK
OR
PA
RI
SC
SD
TN
TX
UT
VT
VA
WA
WV
WI
WY


In [54]:
states=newDF.state.unique()
#states=[13]
df_naics_6=df_naics_6.astype({'relevant_naics': 'string'})
a=county_level(df_naics_2)
b=county_level(df_naics_4)
c=county_level(df_naics_6)
for state in states:
    stateName=stateFips.loc[stateFips.FIPS==state,"Postal Code"].values[0]
    print(stateName)
    a1=a[a.state==state]
    a1.to_csv(f"usa/{stateName}/industries_state{state}_naics2.tsv",sep='\t')
    
    b1=b[b.state==state]
    b1.to_csv(f"usa/{stateName}/industries_state{state}_naics4.tsv",sep='\t')
    
    c1=c[c.state==state]
    c1.to_csv(f"usa/{stateName}/industries_state{state}_naics6.tsv",sep='\t')
    print(c1.dtypes)

GA
id                 object
state               int64
COUNTY              int64
GEO_TTL            object
relevant_naics     string
NAICS2012_TTL      object
NAICS_Sector       object
estab             float64
emp               float64
payann            float64
dtype: object
